# TRI Modeling Validation

gl
<br>
09.29.20

- only three sensors in reasonable proximity for 90-99 and all lead
- want to check the 2000-2018 for potential matches

In [2]:
#Libraries
import pandas as pd
import geopandas as gpd 
import contextily as ctx
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt

In [4]:
#Functions
def haversine(lon1, lat1, lon2, lat2):

    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

## Pulling the sensor data

In [5]:
#Load in TRI data from 1990 - 2018 to look for relevant sensors (run with makefile commands in processed/data_origin.txt)
TRI_base_process_90_18_nopubchem_df = pd.read_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/processed/TRI_base_process_90_18_nopubchem.csv')
TRI_base_process_90_18_nopubchem_df = TRI_base_process_90_18_nopubchem_df.drop(columns = ['Unnamed: 0'])

#While there may be duplicates in the data, we don't need them for this analysis
TRI_base_process_90_18_nopubchem_df = TRI_base_process_90_18_nopubchem_df.drop_duplicates()

#Load in EPA monitors data
EPA_mon = pd.read_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/validation/TRIChemicals_Monitors.csv')

#Interested in the monitors from 1990 to 2018
valid_monitors = EPA_mon[(EPA_mon['first_year']>=1990)]

print('Total TRI releases 1990-2018: {0}'.format(TRI_base_process_90_18_nopubchem_df.shape[0]))
print('\nTotal number of EPA monitors recording after 1990: {0}'.format(valid_monitors.shape[0]))
print('\nEPA tracked chemicals: ')
print(*valid_monitors['chemicalname'].drop_duplicates().values, sep = ", ")
print('\nUnique EPA Sensor Locations: {}'.format(valid_monitors.drop_duplicates(subset= ['latitude','longitude']).shape[0]))

Total TRI releases 1990-2018: 2231

Total number of EPA monitors recording after 1990: 88

EPA tracked chemicals: 
ETHYLBENZENE, STYRENE, 1,2-DIBROMOETHANE, 1,3-BUTADIENE, 1,2-DICHLOROETHANE, METHYL ISOBUTYL KETONE, TETRACHLOROETHYLENE, FORMALDEHYDE, CHLOROFORM, BENZENE, LEAD, NICKEL, CADMIUM, COBALT, DICHLOROMETHANE, ETHYLENE OXIDE, TRICHLOROETHYLENE, NAPHTHALENE, CUMENE

Unique EPA Sensor Locations: 12


In [6]:
#First need to sort the EPA sensors by ID to see which chemicals are at which facilities
valid_monitors['casnumber'] = valid_monitors['casnumber'].str.replace('-','')
epa_sensors_locs = valid_monitors.groupby(['latitude','longitude'])['casnumber'].apply(list)
epa_sensors_locs = epa_sensors_locs.reset_index()

#Gather the CAS numbers for the unique chemicals (EPA sensors)
unique_cas = epa_sensors_locs['casnumber'].to_list()
unique_cas = [item for sublist in unique_cas for item in sublist]
unique_cas =list(dict.fromkeys(unique_cas))

In [7]:
#First calculate the TRI emitters which are closest to the origin source AND have chemicals within the list of TRI emitters 
a= []

for idx in range(epa_sensors_locs.shape[0]):
    locs = TRI_base_process_90_18_nopubchem_df
    temp =epa_sensors_locs.iloc[idx] # This is EPA monitor

    #Should be using Haversin because of the rounded nature of the earth
    locs['haversine_distance_km']=locs.apply(lambda row : haversine(row['LONGITUDE'],row['LATITUDE'],temp['longitude'],temp['latitude']), axis = 1)

    #In order to add multiple entries per each - I think I will just change the iloc here to a boolean based upon distance
    matches = locs[locs['CAS#/COMPOUNDID'].isin(epa_sensors_locs['casnumber'].iloc[idx])]
    matches = matches[matches.haversine_distance_km<50]
    a.append(matches)

nearest_ls = pd.concat([epa_sensors_locs.reset_index(),pd.DataFrame(a).reset_index()],axis=1)
nearest_ls = nearest_ls.drop(columns=['index'])

#Remove any sensors which have no sensors nearby
nearest_ls = nearest_ls[nearest_ls[0].apply(lambda x: x.empty)==False]

In [8]:
#Converting the list of dataframes into one large dataframe
temp_list = []
for rows in range(nearest_ls.shape[0]):
    temp_df = nearest_ls[0].iloc[rows]
    temp_df['EPA_lat'] = nearest_ls['latitude'].iloc[rows]
    temp_df['EPA_long'] = nearest_ls['longitude'].iloc[rows]
    temp_df['casnumber'] = str(nearest_ls['casnumber'].iloc[rows])
    temp_list.append(temp_df)

EPA_TRI_merge_by_nearest_sensor = pd.concat(temp_list)
EPA_TRI_merge_by_nearest_sensor = EPA_TRI_merge_by_nearest_sensor.dropna(subset=['Group'])

In [9]:
#Cleaning up the dataframes a bit 
EPA_TRI_merge_by_nearest_sensor = EPA_TRI_merge_by_nearest_sensor[['FRSID',
                                                                    'YEAR',
                                                                    'TRIFD',
                                                                    'CAS#/COMPOUNDID',
                                                                    'CHEMICAL',
                                                                    'LATITUDE',
                                                                    'LONGITUDE',
                                                                    'EPA_lat',
                                                                    'EPA_long',
                                                                    'casnumber',
                                                                    'haversine_distance_km']]


# Expanding the output so each EPA lat/long ~ sensor ~ showcases the nearest TRI release with distance and years produced
EPA_TRI_merge_by_nearest_sensor_loc_agg = EPA_TRI_merge_by_nearest_sensor.groupby(['EPA_lat','EPA_long','LATITUDE','LONGITUDE','CHEMICAL','TRIFD','haversine_distance_km'])['YEAR'].apply(list).reset_index()
EPA_TRI_merge_by_nearest_sensor_loc_agg = pd.DataFrame(EPA_TRI_merge_by_nearest_sensor_loc_agg)
EPA_TRI_merge_by_nearest_sensor_loc_agg

,EPA_lat,EPA_long,LATITUDE,LONGITUDE,CHEMICAL,TRIFD,haversine_distance_km,YEAR
0,37.198299,-113.1506,37.037627,-113.544195,LEAD,84770STKRP1843E,39.205851,"[2012, 2013, 2014, 2015, 2016, 2017, 2018]"
1,37.198299,-113.1506,37.043001,-113.532888,LEAD,8479WSNRCC1825E,38.040146,[2018]
2,37.198299,-113.1506,37.120370,-113.556790,NICKEL,84770STGRG1301E,37.023942,"[2011, 2012, 2013, 2014, 2015]"
3,37.198299,-113.1506,37.169211,-113.423103,LEAD,8473WSNRCC155NR,24.356513,[2018]
4,37.459080,-113.2251,37.120370,-113.556790,NICKEL,84770STGRG1301E,47.743947,"[2011, 2012, 2013, 2014, 2015]"
...,...,...,...,...,...,...,...,...
593,41.842648,-111.8522,41.763580,-111.860270,LEAD,84321NVRNC1073W,8.817370,"[2010, 2011, 2012, 2013, 2014, 2015, 2016, 201..."
594,41.842648,-111.8522,41.771330,-111.848860,LEAD,8432WGNVRC2151N,7.935030,[2018]
595,41.842648,-111.8522,41.882500,-112.196400,CADMIUM,84330NCRST7285W,28.846324,"[1990, 1991]"
596,41.842648,-111.8522,41.882500,-112.196400,LEAD,84330NCRST7285W,28.846324,"[1990, 1991, 1992, 1993]"


In [10]:
#Saving the Data: 
EPA_TRI_merge_by_nearest_sensor_loc_agg.to_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/validation/EPA_validation_100.csv')

# Modeling EPA Validation Sensors

Joemy merged all EPA sensor data with TRI releases so we can start to validate the model. 

In [11]:
#Load the data
TRI_validation_df = pd.read_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/validation/TRI_ValidationSet.csv')

In [20]:
#Collecting only those simulations through 2014 because that is where I have NARR data through
tri_valid_2014 = TRI_validation_df[TRI_validation_df.year<=2014]

#Extracting the unique ids
stilt_df = tri_valid_2014[['tri_lat','tri_lon','stackheight','sample_dt']].drop_duplicates()
stilt_df.columns = ['lati','long','zagl','run_times']

#Save as csv
stilt_df.to_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/validation/092920_epa_valid_2014.csv',index=False)

In [18]:
stilt_df.shape

(32963, 4)

In [4]:
#So just for sake of efficiency - let's examine just a subset of the data (i choose styrene in 2010)
styrene = TRI_validation_df[(TRI_validation_df.parametername == 'STYRENE') & (TRI_validation_df.year == 2010)]
styrene.describe()

,monitorid,year,latitude,longitude,cas_no,haps_conc,frsid,zip,tri_lat,tri_lon,cascompoundid,stackheight,stackvelocity,stackdiameter
count,342.0,342.0,3.420000e+02,3.420000e+02,342.0,342.000000,3.420000e+02,3.420000e+02,342.000000,342.000000,342.0,342.000000,342.000000,342.000000
mean,490110004.0,2010.0,4.090290e+01,-1.118845e+02,100425.0,0.233781,1.100081e+11,1.402638e+08,40.850004,-112.117043,100425.0,14.766666,9.866667,0.766667
std,0.0,0.0,7.115838e-15,1.423168e-14,0.0,0.672109,1.407497e+07,3.139106e+08,0.126271,0.383589,0.0,9.667929,4.818032,0.335487
min,490110004.0,2010.0,4.090290e+01,-1.118845e+02,100425.0,0.007000,1.100005e+11,8.401600e+04,40.734402,-112.968100,100425.0,7.700000,2.000000,0.300000
25%,490110004.0,2010.0,4.090290e+01,-1.118845e+02,100425.0,0.013000,1.100005e+11,8.402900e+04,40.742111,-112.033600,100425.0,9.400000,6.000000,0.600000
50%,490110004.0,2010.0,4.090290e+01,-1.118845e+02,100425.0,0.079000,1.100008e+11,8.407050e+04,40.816246,-111.942320,100425.0,10.050000,10.200000,0.700000
75%,490110004.0,2010.0,4.090290e+01,-1.118845e+02,100425.0,0.139500,1.100069e+11,8.410400e+04,40.886021,-111.911160,100425.0,15.800000,15.200000,0.900000
max,490110004.0,2010.0,4.090290e+01,-1.118845e+02,100425.0,4.630000,1.100391e+11,8.411623e+08,41.105000,-111.904760,100425.0,35.599998,15.600000,1.400000


Conclusions:

1. Styrene data is available in the years 1996, 1999, 2010-17
2. There is only a single EPA monitor (40.9029, -111.8845)  but four nearby releasing TRI sites
3. Data is available on 57 dates 

In [5]:
trifd_of_interest = styrene['trifd'].drop_duplicates().to_list()
chems_of_interest = styrene['cas_no'].drop_duplicates().to_list()

#Let's filter the data for these trifd's of interest
df = pd.read_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/processed/TRI_valid_2010_2010.csv').drop(columns = 'Unnamed: 0' )

df['TRIFD'] = df.TRIFD.astype('string')
entries_of_interest = df[(df.TRIFD.isin(trifd_of_interest)) & (df.CAS_No.isin(chems_of_interest))]

In [6]:
#Now we should be ready to go
entries_of_interest.to_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/processed/STYRENE_DEMO.csv')

In [41]:
#Which dates do we want to model? (the current setup models on a everyday per year basis)
sorted(styrene['sample_dt'].drop_duplicates())

['01-02-2010',
 '01-08-2010',
 '01-14-2010',
 '01-20-2010',
 '01-26-2010',
 '02-01-2010',
 '02-07-2010',
 '02-13-2010',
 '02-19-2010',
 '02-25-2010',
 '03-03-2010',
 '03-09-2010',
 '03-15-2010',
 '04-02-2010',
 '04-08-2010',
 '04-14-2010',
 '04-20-2010',
 '04-26-2010',
 '05-02-2010',
 '05-08-2010',
 '05-14-2010',
 '05-20-2010',
 '05-26-2010',
 '06-01-2010',
 '06-07-2010',
 '06-13-2010',
 '06-19-2010',
 '06-25-2010',
 '07-01-2010',
 '07-07-2010',
 '07-13-2010',
 '07-19-2010',
 '07-25-2010',
 '07-31-2010',
 '08-12-2010',
 '08-18-2010',
 '08-24-2010',
 '08-30-2010',
 '09-05-2010',
 '09-11-2010',
 '09-17-2010',
 '09-23-2010',
 '09-29-2010',
 '10-05-2010',
 '10-11-2010',
 '10-17-2010',
 '10-23-2010',
 '10-29-2010',
 '11-04-2010',
 '11-10-2010',
 '11-16-2010',
 '11-22-2010',
 '11-28-2010',
 '12-05-2010',
 '12-10-2010',
 '12-16-2010',
 '12-28-2010']

# Run Simulations

In [8]:
a = pd.read_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/processed/stilt_output/shapefile/092520_styrene_test.csv')

In [10]:
a 

,lat,lon,foot,lbsperday,id,TRI_source_lati,TRI_source_long,zagl,Chemical,Release (lbs/year),YEAR,ss_name,ss_path,ss_date
0,40.745,-111.955,8.489833e-02,9.180184e+00,0,40.742110,-111.958020,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,201001060000
1,40.745,-111.955,8.489833e-02,9.180184e+00,0,40.742110,-111.958020,9.4,STYRENE,39468.0,2010,201002060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,201002060000
2,40.745,-111.955,8.489833e-02,9.180184e+00,0,40.742110,-111.958020,9.4,STYRENE,39468.0,2010,201005260000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,201005260000
3,40.745,-111.955,8.489833e-02,9.180184e+00,0,40.742110,-111.958020,9.4,STYRENE,39468.0,2010,201001290000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,201001290000
4,40.755,-111.955,7.312495e-02,7.907112e+00,0,40.742110,-111.958020,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,201001060000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3503612,42.055,-109.045,8.670778e-07,1.187778e-08,7,40.886022,-111.904759,0.0,STYRENE,5.0,2010,201007200000_-111.904759_40.886022_0_foot,data/processed/stilt_output/netcdf/092520_styr...,201007200000
3503613,42.055,-109.035,8.645237e-07,1.184279e-08,7,40.886022,-111.904759,0.0,STYRENE,5.0,2010,201007200000_-111.904759_40.886022_0_foot,data/processed/stilt_output/netcdf/092520_styr...,201007200000
3503614,42.055,-109.025,8.593766e-07,1.177228e-08,7,40.886022,-111.904759,0.0,STYRENE,5.0,2010,201007200000_-111.904759_40.886022_0_foot,data/processed/stilt_output/netcdf/092520_styr...,201007200000
3503615,42.055,-109.015,8.519328e-07,1.167031e-08,7,40.886022,-111.904759,0.0,STYRENE,5.0,2010,201007200000_-111.904759_40.886022_0_foot,data/processed/stilt_output/netcdf/092520_styr...,201007200000


In [2]:
#Let's take a look: 
styrene_gdf = gpd.read_file('/home/boogie2/Hanson_Lab/TRI_STILT/data/processed/stilt_output/shapefile/092520_styrene')
styrene_gdf['ss_date'] = pd.to_datetime(styrene_gdf['ss_date'])

In [3]:
styrene_gdf.shape

(38832, 15)

In [42]:
temp_20100106 = styrene_gdf[styrene_gdf['ss_date']== '2010-01-06']
fig, ax = plt.subplots(figsize=(15,15))
temp_20100106[temp_20100106.lbsperday>0.1].plot(column = 'lbsperday',ax = ax,alpha = 0.5,markersize=5)
ctx.add_basemap(ax=ax)

plt.close()

In [38]:
styrene_gdf[['TRI_source','TRI_sour_1']].drop_duplicates()

,TRI_source,TRI_sour_1
0,40.742110,-111.958020
14992,40.786392,-111.911163
25782,41.105000,-112.033600
28400,40.734400,-112.968100
32116,40.886022,-111.904759
32168,40.846100,-111.926620


In [37]:
temp_20100106

,lat,lon,foot,lbsperday,id,TRI_source,TRI_sour_1,zagl,Chemical,Release (l,YEAR,ss_name,ss_path,ss_date,geometry
0,40.745,-111.955,0.084898,9.180184,0,40.74211,-111.95802,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,2010-01-06,POINT (-12462773.592 4974801.670)
4,40.755,-111.955,0.073125,7.907112,0,40.74211,-111.95802,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,2010-01-06,POINT (-12462773.592 4976271.108)
8,40.755,-111.945,0.069022,7.463464,0,40.74211,-111.95802,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,2010-01-06,POINT (-12461660.397 4976271.108)
12,40.765,-111.945,0.147847,15.986922,0,40.74211,-111.95802,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,2010-01-06,POINT (-12461660.397 4977740.767)
16,40.765,-111.935,0.003443,0.372280,0,40.74211,-111.95802,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,2010-01-06,POINT (-12460547.202 4977740.767)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14972,40.945,-112.365,0.001034,0.111794,0,40.74211,-111.95802,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,2010-01-06,POINT (-12508414.583 5004232.558)
14976,40.945,-112.355,0.001060,0.114596,0,40.74211,-111.95802,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,2010-01-06,POINT (-12507301.388 5004232.558)
14980,40.945,-112.345,0.001065,0.115107,0,40.74211,-111.95802,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,2010-01-06,POINT (-12506188.193 5004232.558)
14984,40.945,-112.335,0.001046,0.113152,0,40.74211,-111.95802,9.4,STYRENE,39468.0,2010,201001060000_-111.95802_40.74211_9.4_foot,data/processed/stilt_output/netcdf/092520_styr...,2010-01-06,POINT (-12505074.998 5004232.558)
